In [8]:
# Default Setup Cell.  
# It imports environment variables, define 'devtools.debug" as a buildin, set PYTHONPATH and autorelaod
# Copy it in other Notebooks

from dotenv import load_dotenv
from devtools import debug
import builtins

setattr(builtins, "debug", debug)
load_dotenv(verbose=True)

%load_ext autoreload
%autoreload 2

!export PYTHONPATH=":./python"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from python.ai_core.llm import LlmFactory, get_llm

LlmFactory(llm_id="gpt_4o_openai").get_litellm_model_name()



'gpt-4o'

In [ ]:
import site; print(site.getsitepackages())